# Integration weights training set

This notebooks prepares the training set for obtaining integration weights.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [ ]:
# Loading data
integration_weights_file = r'./Data/integration_weights.hdf5'
with h5py.File(integration_weights_file, 'r') as hf:
    durations = np.array(hf['durations'][:])
    res_1_iw_I = np.array(hf['res_1_iw_I'][:])
    res_1_iw_Q = np.array(hf['res_1_iw_Q'][:])
    res_2_iw_I = np.array(hf['res_2_iw_I'][:])
    res_2_iw_Q = np.array(hf['res_2_iw_Q'][:])

sample_file = r'./Data/res_1_IJK_sample.hdf5'
with h5py.File(sample_file, 'r') as hf:
    res_1_IJK_sample = np.array(hf['res_1_IJK_sample'][:])

In [ ]:
IData_sample = (res_1_IJK_sample @ res_1_iw_I[:3, :]).reshape((-1, 2, 24))
QData_sample = (res_1_IJK_sample @ res_1_iw_Q[:3, :]).reshape((-1, 2, 24))
IData_sample = np.repeat(IData_sample, 10, axis=0)
QData_sample = np.repeat(QData_sample, 10, axis=0)

# Add Gaussian noise to simulate the ADC record
I_noise = np.random.normal(scale=1e-2, size=IData_sample.shape)
Q_noise = np.random.normal(scale=1e-2, size=IData_sample.shape)

IQ_noise = np.concatenate([I_noise, Q_noise], axis=2)
res_1_iw_IQ = np.hstack([res_1_iw_I, res_1_iw_Q])

IQ_noise -= np.einsum(res_1_iw_IQ[:2], [0, 1], res_1_iw_IQ[:2] / 48.0, [0, 2], IQ_noise, [3, 4, 1], [3, 4, 2])

IData_sample += IQ_noise[:, :, :24]
QData_sample += IQ_noise[:, :, 24:]

In [5]:
ADC_sample_file = r'./Data/ADC_sample.hdf5'
with h5py.File(ADC_sample_file, 'w') as hf:
    hf.create_dataset('IData_sample', data=IData_sample)
    hf.create_dataset('QData_sample', data=QData_sample)